In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract
from pyspark.sql.functions import explode, split


In [0]:
df = spark.read.csv("dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_commentary_data-1.csv", header=True, inferSchema=True)
df.show()


+----+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+-----------+-------------+-----------+-------------+-------------+-------------+--------------------+-------------------+-------------+-------------+--------------------+-------------------+-------+-------+--------------------+
|year|series_type|         series_name| match_no|match_type|match_id|         match_venue|match_status|  match_winning_team|match_tie_breaker|          match_toss|             umpires|       match_referee|       third_umpires|      match_datetime|team1_name|team2_name|team1_score|team1_wickets|team2_score|team2_wickets|team1_captain|team1_players|         team1_bench|team1_support_staff|team2_captain|team2_players|         team2_bench|team2_support_staff|ball_no|over_no|     ball_commentary|
+----+

In [0]:
dbutils.fs.ls("dbfs:/FileStore/shared_uploads/girish792004@gmail.com/")


Out[2]: [FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_commentary_data-1.csv', name='ipl_commentary_data-1.csv', size=147312850, modificationTime=1745389335000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_commentary_data.csv', name='ipl_commentary_data.csv', size=147312850, modificationTime=1745295584000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_over_data-1.csv', name='ipl_over_data-1.csv', size=20877706, modificationTime=1745384072000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_over_data-2.csv', name='ipl_over_data-2.csv', size=20877706, modificationTime=1745384105000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_over_data-3.csv', name='ipl_over_data-3.csv', size=20877706, modificationTime=1745389224000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/girish792004@gmail.com/ipl_over_data.csv', name='ipl_over_data.csv', size=20877706,

In [0]:
from pyspark.sql.functions import array, explode, col, trim

teams_df = df.select("year", trim("team1_name").alias("team1"), trim("team2_name").alias("team2")) \
    .selectExpr("year", "team1", "team2") \
    .withColumn("team", explode(array("team1", "team2"))) \
    .drop("team1", "team2") \
    .dropna(subset=["team"]) \
    .distinct() \
    .groupBy("year") \
    .count() \
    .withColumnRenamed("count", "number_of_teams")

display(teams_df)


year,number_of_teams
2020,8
2019,8
2017,8
2018,8
2022,10
2025,10
2023,10
2021,9
2024,10


In [0]:
match_status_df = df.groupBy("match_status").count()
display(match_status_df)


match_status,count
completed,142630
null,9
tied,1975


In [0]:
from pyspark.sql.functions import avg, countDistinct

avg_balls = df.groupBy("match_id").agg(countDistinct("ball_no").alias("balls_per_match"))
avg_result = avg_balls.select(avg("balls_per_match").alias("avg_balls_per_match"))

display(avg_result)


avg_balls_per_match
124.40380952380953


In [0]:
wins_df = df.groupBy("year", "match_winning_team") \
            .count() \
            .withColumnRenamed("count", "matches_won") \
            .orderBy("year", "matches_won", ascending=[True, False])

display(wins_df)


year,match_winning_team,matches_won
". Some real confused umpiring here. The same line a couple of overs go wasn't a wide. Samson's not impressed; and he's right in the umpire's ear""",null,1
2017,Mumbai Indians,2625
2017,Rising Pune Supergiant,2346
2017,Kolkata Knight Riders,1963
2017,Sunrisers Hyderabad,1914
2017,Kings XI Punjab,1604
2017,Delhi Daredevils,1440
2017,Gujarat Lions,924
2017,Royal Challengers Bangalore,746
2017,Match tied ( Mumbai Indians,250


In [0]:
from pyspark.sql.functions import regexp_extract, col, when
df_extracted = df.withColumn("bowler", regexp_extract(col("ball_commentary"), r"^([^ ]+) to", 1)) \
                 .withColumn("batsman", regexp_extract(col("ball_commentary"), r"to ([^,]+)", 1)) \
                 .withColumn("runs_text", regexp_extract(col("ball_commentary"), r",\s*([A-Z ]+|\d+ run|FOUR|SIX|no run|1 run|2 runs|3 runs)", 1)) \
                 .withColumn("runs", when(col("runs_text").rlike("no run"), 0)
                                   .when(col("runs_text").rlike("FOUR"), 4)
                                   .when(col("runs_text").rlike("SIX"), 6)
                                   .when(col("runs_text").rlike("1 run"), 1)
                                   .when(col("runs_text").rlike("2 runs"), 2)
                                   .when(col("runs_text").rlike("3 runs"), 3)
                                   .otherwise(0)) \
                 .select("match_id", "ball_no", "ball_commentary", "bowler", "batsman", "runs")


In [0]:
from pyspark.sql.functions import collect_list, collect_set
batsmen_per_team = df.select("match_id", "team1_name", "team2_name") \
    .join(df_extracted, on="match_id", how="inner")

batsmen_summary = batsmen_per_team.groupBy("match_id") \
    .agg(
        collect_list("batsman").alias("batsmen_list"),
        collect_set("batsman").alias("batsmen_set")
    )

display(batsmen_summary)
